In [12]:
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
# Load Packages
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

import os

In [13]:
dir_ = '../../../Dataset/FNS2022/Spanish/validation/'
annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
lex_rank_dir = 'lex_rank_summary_es'
text_rank_dir = 'text_rank_summary_es'

In [14]:
os.makedirs(text_rank_dir)

FileExistsError: [Errno 17] File exists: 'text_rank_summary_es'

In [9]:
num_file = 0

for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        print("Processing File Number: ", num_file, file)
        num_file = num_file +1 
        file_id = file.split('.')[0]
        
        tmp_file = []
        with open(os.path.join(dir_, annual_reports_dir, str(file_id) + '.txt'), "r", encoding='utf-8') as report_file:
            for line in report_file:
                line = line.replace("\n", "").replace("\t", " ").replace("\xa0", " ")
                tmp_file.append(line)
        report_txt = " ".join(tmp_file)
        
        # For Strings
        parser = PlaintextParser.from_string(report_txt,Tokenizer("spanish"))
        # Summarize using sumy TextRank
        summarizer = TextRankSummarizer()
        summary =summarizer(parser.document,20)
        text_summary=""
        for sentence in summary:
            text_summary+=str(sentence)
        summary_split = text_summary.split(' ')
        number_of_words = len(summary_split)
        print(number_of_words)
        if number_of_words > 1000:
            text_summary = " ".join(summary_split[:1000])
        
        with open(os.path.join(text_rank_dir, file_id+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(text_summary))
            
        if ".DS_Store" in file:
            continue
            
    except Exception as e:
        print(file, e)
        

Processing File Number:  0 0007.txt
1979
Processing File Number:  1 0013.txt
1563
Processing File Number:  2 0012.txt
1972
Processing File Number:  3 0006.txt


KeyboardInterrupt: 

In [10]:
os.makedirs(lex_rank_dir)

In [11]:
num_file = 0

for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        print("Processing File Number: ", num_file, file)
        num_file = num_file +1 
        file_id = file.split('.')[0]
        
        tmp_file = []
        with open(os.path.join(dir_, annual_reports_dir, str(file_id) + '.txt'), "r", encoding='utf-8') as report_file:
            for line in report_file:
                line = line.replace("\n", "").replace("\t", " ").replace("\xa0", " ")
                tmp_file.append(line)
        report_txt = " ".join(tmp_file)
        
        # For Strings
        parser = PlaintextParser.from_string(report_txt,Tokenizer("spanish"))
        # Summarize using sumy LexRank
        summarizer = LexRankSummarizer()
        summary =summarizer(parser.document,20)
        text_summary=""
        for sentence in summary:
            text_summary+=str(sentence)
        summary_split = text_summary.split(' ')
        number_of_words = len(summary_split)
        print(number_of_words)
        if number_of_words > 1000:
            text_summary = " ".join(summary_split[:1000])
        
        with open(os.path.join(lex_rank_dir, file_id+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(text_summary))
            
        if ".DS_Store" in file:
            continue
            
    except Exception as e:
        print(file, e)

Processing File Number:  0 0007.txt
1056
Processing File Number:  1 0013.txt
907
Processing File Number:  2 0012.txt
1006
Processing File Number:  3 0006.txt


KeyboardInterrupt: 

In [15]:
from rouge_evaluation import get_rouge_scores
gold_summary_dir_ = os.path.join(dir_, gold_summary_dir)

In [27]:
system_summary_dir = text_rank_dir

In [28]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'])

rouge_scores = {}
rouge_scores['rouge-1'] = {}
rouge_scores['rouge-2'] = {}

rouge_scores['rouge-1']['p'] = 0
rouge_scores['rouge-1']['r'] = 0
rouge_scores['rouge-1']['f'] = 0
rouge_scores['rouge-2']['p'] = 0
rouge_scores['rouge-2']['r'] = 0
rouge_scores['rouge-2']['f'] = 0
num_file = 0

for file in os.listdir(system_summary_dir):
    try:
        if num_file%10 == 0:
            print("Processing File Number: ", num_file)
        file_id = file.split('.')[0]
        if ".DS_Store" in file:
            continue
            
        with open(os.path.join(system_summary_dir,file), "r",  encoding='utf-8') as f:
            system_summary_txt = f.read()
            print(file, 'summary len: ', len(system_summary_txt.split(' ')))
                
        sum_rouge_scores = {}
        sum_rouge_scores['rouge-1'] = {}
        sum_rouge_scores['rouge-2'] = {}
        sum_rouge_scores['rouge-L'] = {}

        sum_rouge_scores['rouge-1']['p'] = 0
        sum_rouge_scores['rouge-1']['r'] = 0
        sum_rouge_scores['rouge-1']['f'] = 0
        sum_rouge_scores['rouge-2']['p'] = 0
        sum_rouge_scores['rouge-2']['r'] = 0
        sum_rouge_scores['rouge-2']['f'] = 0
        
        num_sum_file = 0
        
        for sum_file in os.listdir(gold_summary_dir_):
            if ".DS_Store" in file:
                continue
            if os.path.isfile(os.path.join(gold_summary_dir_,sum_file)) and file_id==sum_file.split("_")[0]:
                #print(file_id)
                
                with open(os.path.join(gold_summary_dir_,sum_file), "r",  encoding='utf-8') as f:
                    gold_sum_txt = f.read()
                    sum_scores = scorer.score(gold_sum_txt, system_summary_txt)
                    sum_rouge_scores['rouge-1']['p'] = sum_rouge_scores['rouge-1']['p'] + sum_scores['rouge1'][0]
                    sum_rouge_scores['rouge-1']['r'] = sum_rouge_scores['rouge-1']['r'] + sum_scores['rouge1'][1]
                    sum_rouge_scores['rouge-1']['f'] = sum_rouge_scores['rouge-1']['f'] + sum_scores['rouge1'][2]
                    sum_rouge_scores['rouge-2']['p'] = sum_rouge_scores['rouge-2']['p'] + sum_scores['rouge2'][0]
                    sum_rouge_scores['rouge-2']['r'] = sum_rouge_scores['rouge-2']['r'] + sum_scores['rouge2'][1]
                    sum_rouge_scores['rouge-2']['f'] = sum_rouge_scores['rouge-2']['f'] + sum_scores['rouge2'][2]
                    num_sum_file = num_sum_file + 1
            
        sum_rouge_scores['rouge-1']['p'] = sum_rouge_scores['rouge-1']['p'] / (num_sum_file)
        sum_rouge_scores['rouge-1']['r'] = sum_rouge_scores['rouge-1']['r'] / (num_sum_file)
        sum_rouge_scores['rouge-1']['f'] = sum_rouge_scores['rouge-1']['f'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['p'] = sum_rouge_scores['rouge-2']['p'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['r'] = sum_rouge_scores['rouge-2']['r'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['f'] = sum_rouge_scores['rouge-2']['f'] / (num_sum_file)
        
        
        rouge_scores['rouge-1']['p'] = rouge_scores['rouge-1']['p'] + sum_rouge_scores['rouge-1']['p']
        rouge_scores['rouge-1']['r'] = rouge_scores['rouge-1']['r'] + sum_rouge_scores['rouge-1']['r']
        rouge_scores['rouge-1']['f'] = rouge_scores['rouge-1']['f'] + sum_rouge_scores['rouge-1']['f']
        rouge_scores['rouge-2']['p'] = rouge_scores['rouge-2']['p'] + sum_rouge_scores['rouge-2']['p']
        rouge_scores['rouge-2']['r'] = rouge_scores['rouge-2']['r'] + sum_rouge_scores['rouge-2']['r']
        rouge_scores['rouge-2']['f'] = rouge_scores['rouge-2']['f'] + sum_rouge_scores['rouge-2']['f']
        
        num_file = num_file + 1
        
        
    except Exception as e:
        print(e)
        pass

Processing File Number:  0
0007.txt summary len:  1000
0013.txt summary len:  1000
0012.txt summary len:  1000


In [29]:
num_file

3

In [30]:
rouge_scores['rouge-1']['p'] = rouge_scores['rouge-1']['p'] / (num_file)
rouge_scores['rouge-1']['r'] = rouge_scores['rouge-1']['r'] / (num_file)
rouge_scores['rouge-1']['f'] = rouge_scores['rouge-1']['f'] / (num_file)
rouge_scores['rouge-2']['p'] = rouge_scores['rouge-2']['p'] / (num_file)
rouge_scores['rouge-2']['r'] = rouge_scores['rouge-2']['r'] / (num_file)
rouge_scores['rouge-2']['f'] = rouge_scores['rouge-2']['f'] / (num_file)
rouge_scores

{'rouge-1': {'p': 0.24506933862890615,
  'r': 0.5427214854650597,
  'f': 0.31913097841036675},
 'rouge-2': {'p': 0.06567242634562602,
  'r': 0.14262845695239368,
  'f': 0.08462920416614424}}

lexrank

{'rouge-1': {'p': 0.26313257359844694,
  'r': 0.5441491830866335,
  'f': 0.33378882462752263},
 'rouge-2': {'p': 0.07381182375468719,
  'r': 0.14377406029715936,
  'f': 0.0916599937672229}}
  
 
textrank

{'rouge-1': {'p': 0.24506933862890615,
  'r': 0.5427214854650597,
  'f': 0.31913097841036675},
 'rouge-2': {'p': 0.06567242634562602,
  'r': 0.14262845695239368,
  'f': 0.08462920416614424}}

